In [1]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import os
import random

In [2]:
batch_size=512
epochs=300
seed=1
cuda=False and torch.cuda.is_available()
log_interval=1
r_dim=128
z_dim=128
result_path="results_np_rz_y_hat/"

In [3]:

torch.manual_seed(seed)
random.seed(seed)
device = torch.device("cpu") #"cuda" if args.cuda else 

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)


In [4]:
def get_context_idx(N):
    # generate the indeces of the N context points in a flattened image
    idx = random.sample(range(0, 784), N)
    idx = torch.tensor(idx, device=device)
    return idx
def generate_grid(h, w):
    rows = torch.linspace(0, 1, h, device=device)
    cols = torch.linspace(0, 1, w, device=device)
    grid = torch.stack([cols.repeat(h, 1).t().contiguous().view(-1), rows.repeat(w)], dim=1)
    grid = grid.unsqueeze(0)
    return grid
def idx_to_y(idx, data):
    # get the [0;1] pixel intensity at each index
    y = torch.index_select(data, dim=1, index=idx)
    return y
def idx_to_x(idx, batch_size):
    # From flat idx to 2d coordinates of the 28x28 grid. E.g. 35 -> (1, 7)
    # Equivalent to np.unravel_index()
    x = torch.index_select(x_grid, dim=1, index=idx)
    x = x.expand(batch_size, -1, -1)
    return x

In [ ]:
class NP(nn.Module):
    def __init__(self, r_dim,z_dim):
        super(NP, self).__init__()
        self.r_dim = r_dim
        self.z_dim = z_dim
        
        self.h_1 = nn.Linear(3, 400)
        self.h_2 = nn.Linear(400, 400)
        self.h_3 = nn.Linear(400, self.r_dim)

        self.latent_layer_1=nn.Linear(3,400)
        self.latent_layer_2=nn.Linear(400,400)
        self.latent_layer_3=nn.Linear(400,self.r_dim)
        
        self.s_to_z_mean = nn.Linear(self.r_dim, self.z_dim)
        self.s_to_z_logvar = nn.Linear(self.r_dim, self.z_dim)

        self.g_1 = nn.Linear(self.r_dim+ self.z_dim + 2, 400)
        self.g_2 = nn.Linear(400,400)
        self.g_3 = nn.Linear(400,400)
        self.g_y = nn.Linear(400, 1)
        

    def h(self, x_y):
        x_y = F.relu(self.h_1(x_y))
        x_y = F.relu(self.h_2(x_y))
        x_y = F.relu(self.h_3(x_y))
        return x_y

    def latent(self, x_y):
        x_y = F.relu(self.latent_layer_1(x_y))
        x_y = F.relu(self.latent_layer_2(x_y))
        x_y = F.relu(self.latent_layer_3(x_y))
        return x_y

    def aggregate(self, r):
        return torch.mean(r, dim=1)

    def reparameterise(self, z):
        mu, std= z
        eps = torch.randn_like(std)
        z_sample = eps.mul(std).add_(mu)
        z_sample = z_sample.unsqueeze(1).expand(-1, 784, -1)
        return z_sample

    def g(self, r,z_sample, x_target):
        rz_et_x = torch.cat([r,z_sample, x_target], dim=2)
        input = F.relu(self.g_1(rz_et_x))
        input = F.relu(self.g_2(input))
        input = F.relu(self.g_3(input))
        y_hat=torch.sigmoid(self.g_y(input))
        return y_hat
    
    def xy_to_r_params(self, x, y):       
        x_y = torch.cat([x, y], dim=2)
        r_i = self.h(x_y)
        r = self.aggregate(r_i)
        return r
    
    def xy_to_z_params(self, x, y):
        
        x_y = torch.cat([x, y], dim=2)
        
        s_i = self.latent(x_y)
        s = self.aggregate(s_i)
        mu = self.s_to_z_mean(s)
        logvar = self.s_to_z_logvar(s)
        sigma=0.1+0.9*F.softplus(logvar)
        return mu, sigma

    def forward(self, x_context, y_context, x_all=None, y_all=None):
        #produire r
        r_context = self.xy_to_r_params(x_context, y_context)
        r_expand = r_context.unsqueeze(1).expand(-1, 784, -1)
        #produire z
        z_context = self.xy_to_z_params(x_context, y_context)  # (mu, logvar) of z
        if self.training:  # loss function will try to keep z_context close to z_all         
            z_all = self.xy_to_z_params(x_all, y_all)  
        else:  # at test time we don't have the image so we use only the context
            z_all = z_context
        z_sample = self.reparameterise(z_all)
        
        # reconstruct the whole image including the provided context points
        x_target = x_grid.expand(y_context.shape[0], -1, -1)
        
        y_hat = self.g(r_expand,z_sample, x_target)
    
        
        return y_hat,z_all, z_context

In [6]:
def kl_div_gaussians(mu_q, logvar_q, mu_p, logvar_p):
    var_p = torch.exp(logvar_p)
    kl_div = (torch.exp(logvar_q) + (mu_q - mu_p) ** 2) / var_p \
             - 1.0 \
             + logvar_p - logvar_q
    kl_div = 0.5 * kl_div.sum()
    return kl_div


def np_loss(y_hat, y, z_all, z_context):
    # get log probability
    #logprob=-(y - mu)**2 / (2 * sigma**2) - torch.log(sigma)
    logprob=F.binary_cross_entropy(y_hat, y, reduction="sum")
    # get KL divergence between prior and posterior
    KLD = kl_div_gaussians(z_all[0], z_all[1], z_context[0], z_context[1])
    # maximize prob and minimize KL divergence
    return logprob + KLD


In [7]:
model = NP(r_dim,z_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=5e-4)
x_grid = generate_grid(28, 28)
os.makedirs(result_path, exist_ok=True)

In [8]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (y_all, _) in enumerate(train_loader):
        
        batch_size = y_all.shape[0]
        
        y_all = y_all.to(device).view(batch_size, -1, 1)
        
        N = random.randint(1, 784)  # number of context points
        context_idx = get_context_idx(N)
        x_context = idx_to_x(context_idx, batch_size)
        y_context = idx_to_y(context_idx, y_all)
        x_all = x_grid.expand(batch_size, -1, -1)

        optimizer.zero_grad()
        
        y_hat, z_all, z_context = model(x_context, y_context, x_all, y_all)
        
        loss = np_loss(y_hat, y_all, z_all, z_context).sum(dim=0).mean()
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(y_all), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader),
                       loss.item() / len(y_all)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))

In [11]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (y_all, _) in enumerate(test_loader):
            y_all = y_all.to(device).view(y_all.shape[0], -1, 1)
            batch_size = y_all.shape[0]

            N = 300
            context_idx = get_context_idx(N)
            x_context = idx_to_x(context_idx, batch_size)
            y_context = idx_to_y(context_idx, y_all)
            
            y_hat, z_all, z_context = model(x_context, y_context)
            test_loss += np_loss(y_hat, y_all, z_all, z_context).sum(dim=0).mean().item()

            if i == 0:  # save PNG of reconstructed examples
                plot_Ns = [10, 100, 300, 784]
                num_examples = min(batch_size, 16)
                for N in plot_Ns:
                    recons = []
                    context_idx = get_context_idx(N)
                    x_context = idx_to_x(context_idx, batch_size)
                    y_context = idx_to_y(context_idx, y_all)
                    for d in range(20):
                        y_hat, _, _ = model(x_context, y_context)
                        recons.append(y_hat[:num_examples])     
                    recons = torch.cat(recons).view(-1, 1, 28, 28).expand(-1, 3, -1, -1)
                    background = torch.tensor([0., 0., 1.], device=device)
                    background = background.view(1, -1, 1).expand(num_examples, 3, 784).contiguous()
                    context_pixels = y_all[:num_examples].view(num_examples, 1, -1)[:, :, context_idx]
                    context_pixels = context_pixels.expand(num_examples, 3, -1)
                    background[:, :, context_idx] = context_pixels
                    comparison = torch.cat([background.view(-1, 3, 28, 28),
                                            recons])
                    save_image(comparison.cpu(),
                               result_path+"ep_" + str(epoch) +
                               "_nps_" + str(N) + ".png", nrow=num_examples)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [16]:
for epoch in range(2, epochs + 1):
    train(epoch)
    test(epoch)

Train Epoch: 2 [0/60000 (0%)]	Loss: 294.924866
Train Epoch: 2 [256/60000 (0%)]	Loss: 299.187164
Train Epoch: 2 [512/60000 (1%)]	Loss: 297.817963
Train Epoch: 2 [768/60000 (1%)]	Loss: 297.053925
Train Epoch: 2 [1024/60000 (2%)]	Loss: 298.183746
Train Epoch: 2 [1280/60000 (2%)]	Loss: 291.166138
Train Epoch: 2 [1536/60000 (3%)]	Loss: 300.064514
Train Epoch: 2 [1792/60000 (3%)]	Loss: 303.113098
Train Epoch: 2 [2048/60000 (3%)]	Loss: 305.715881
Train Epoch: 2 [2304/60000 (4%)]	Loss: 297.153625
Train Epoch: 2 [2560/60000 (4%)]	Loss: 299.392609
Train Epoch: 2 [2816/60000 (5%)]	Loss: 302.224060
Train Epoch: 2 [3072/60000 (5%)]	Loss: 302.960052
Train Epoch: 2 [3328/60000 (6%)]	Loss: 298.285950
Train Epoch: 2 [3584/60000 (6%)]	Loss: 300.708435
Train Epoch: 2 [3840/60000 (6%)]	Loss: 294.541687
Train Epoch: 2 [4096/60000 (7%)]	Loss: 301.471313
Train Epoch: 2 [4352/60000 (7%)]	Loss: 298.001282
Train Epoch: 2 [4608/60000 (8%)]	Loss: 298.316254
Train Epoch: 2 [4864/60000 (8%)]	Loss: 297.637695
Train 

Train Epoch: 2 [40704/60000 (68%)]	Loss: 244.881607
Train Epoch: 2 [40960/60000 (68%)]	Loss: 242.330917
Train Epoch: 2 [41216/60000 (69%)]	Loss: 238.309296
Train Epoch: 2 [41472/60000 (69%)]	Loss: 242.943863
Train Epoch: 2 [41728/60000 (69%)]	Loss: 241.912140
Train Epoch: 2 [41984/60000 (70%)]	Loss: 238.234177
Train Epoch: 2 [42240/60000 (70%)]	Loss: 246.562119
Train Epoch: 2 [42496/60000 (71%)]	Loss: 235.569916
Train Epoch: 2 [42752/60000 (71%)]	Loss: 242.612045
Train Epoch: 2 [43008/60000 (71%)]	Loss: 245.293228
Train Epoch: 2 [43264/60000 (72%)]	Loss: 239.151398
Train Epoch: 2 [43520/60000 (72%)]	Loss: 241.227356
Train Epoch: 2 [43776/60000 (73%)]	Loss: 250.912109
Train Epoch: 2 [44032/60000 (73%)]	Loss: 250.866531
Train Epoch: 2 [44288/60000 (74%)]	Loss: 246.352798
Train Epoch: 2 [44544/60000 (74%)]	Loss: 240.547165
Train Epoch: 2 [44800/60000 (74%)]	Loss: 241.756226
Train Epoch: 2 [45056/60000 (75%)]	Loss: 238.434387
Train Epoch: 2 [45312/60000 (75%)]	Loss: 241.269028
Train Epoch:

Train Epoch: 3 [20992/60000 (35%)]	Loss: 204.643723
Train Epoch: 3 [21248/60000 (35%)]	Loss: 200.999481
Train Epoch: 3 [21504/60000 (36%)]	Loss: 198.534760
Train Epoch: 3 [21760/60000 (36%)]	Loss: 196.812668
Train Epoch: 3 [22016/60000 (37%)]	Loss: 202.343201
Train Epoch: 3 [22272/60000 (37%)]	Loss: 201.160477
Train Epoch: 3 [22528/60000 (37%)]	Loss: 203.225723
Train Epoch: 3 [22784/60000 (38%)]	Loss: 204.332336
Train Epoch: 3 [23040/60000 (38%)]	Loss: 206.204193
Train Epoch: 3 [23296/60000 (39%)]	Loss: 206.469452
Train Epoch: 3 [23552/60000 (39%)]	Loss: 211.160187
Train Epoch: 3 [23808/60000 (40%)]	Loss: 200.469025
Train Epoch: 3 [24064/60000 (40%)]	Loss: 200.808563
Train Epoch: 3 [24320/60000 (40%)]	Loss: 201.310028
Train Epoch: 3 [24576/60000 (41%)]	Loss: 197.369141
Train Epoch: 3 [24832/60000 (41%)]	Loss: 209.259338
Train Epoch: 3 [25088/60000 (42%)]	Loss: 206.021866
Train Epoch: 3 [25344/60000 (42%)]	Loss: 200.415131
Train Epoch: 3 [25600/60000 (43%)]	Loss: 209.325867
Train Epoch:

Train Epoch: 4 [1024/60000 (2%)]	Loss: 203.591751
Train Epoch: 4 [1280/60000 (2%)]	Loss: 194.644409
Train Epoch: 4 [1536/60000 (3%)]	Loss: 201.215836
Train Epoch: 4 [1792/60000 (3%)]	Loss: 198.786972
Train Epoch: 4 [2048/60000 (3%)]	Loss: 206.166168
Train Epoch: 4 [2304/60000 (4%)]	Loss: 204.242355
Train Epoch: 4 [2560/60000 (4%)]	Loss: 203.658203
Train Epoch: 4 [2816/60000 (5%)]	Loss: 194.783768
Train Epoch: 4 [3072/60000 (5%)]	Loss: 200.852066
Train Epoch: 4 [3328/60000 (6%)]	Loss: 202.721771
Train Epoch: 4 [3584/60000 (6%)]	Loss: 201.910797
Train Epoch: 4 [3840/60000 (6%)]	Loss: 198.283432
Train Epoch: 4 [4096/60000 (7%)]	Loss: 198.622803
Train Epoch: 4 [4352/60000 (7%)]	Loss: 197.833633
Train Epoch: 4 [4608/60000 (8%)]	Loss: 196.667847
Train Epoch: 4 [4864/60000 (8%)]	Loss: 196.926224
Train Epoch: 4 [5120/60000 (9%)]	Loss: 199.129791
Train Epoch: 4 [5376/60000 (9%)]	Loss: 198.720367
Train Epoch: 4 [5632/60000 (9%)]	Loss: 199.465103
Train Epoch: 4 [5888/60000 (10%)]	Loss: 196.752029

Train Epoch: 4 [41728/60000 (69%)]	Loss: 197.558411
Train Epoch: 4 [41984/60000 (70%)]	Loss: 199.208557
Train Epoch: 4 [42240/60000 (70%)]	Loss: 200.189743
Train Epoch: 4 [42496/60000 (71%)]	Loss: 198.187531
Train Epoch: 4 [42752/60000 (71%)]	Loss: 201.827667
Train Epoch: 4 [43008/60000 (71%)]	Loss: 194.170197
Train Epoch: 4 [43264/60000 (72%)]	Loss: 198.362915
Train Epoch: 4 [43520/60000 (72%)]	Loss: 196.151306
Train Epoch: 4 [43776/60000 (73%)]	Loss: 199.990234
Train Epoch: 4 [44032/60000 (73%)]	Loss: 204.308426
Train Epoch: 4 [44288/60000 (74%)]	Loss: 205.088394
Train Epoch: 4 [44544/60000 (74%)]	Loss: 204.480972
Train Epoch: 4 [44800/60000 (74%)]	Loss: 197.238800
Train Epoch: 4 [45056/60000 (75%)]	Loss: 194.426056
Train Epoch: 4 [45312/60000 (75%)]	Loss: 202.977982
Train Epoch: 4 [45568/60000 (76%)]	Loss: 201.398346
Train Epoch: 4 [45824/60000 (76%)]	Loss: 197.908340
Train Epoch: 4 [46080/60000 (77%)]	Loss: 197.742218
Train Epoch: 4 [46336/60000 (77%)]	Loss: 199.685593
Train Epoch:

...

Train Epoch: 155 [1792/60000 (3%)]	Loss: 97.867874
Train Epoch: 155 [2048/60000 (3%)]	Loss: 89.530067
Train Epoch: 155 [2304/60000 (4%)]	Loss: 86.110222
Train Epoch: 155 [2560/60000 (4%)]	Loss: 143.914917
Train Epoch: 155 [2816/60000 (5%)]	Loss: 101.337250
Train Epoch: 155 [3072/60000 (5%)]	Loss: 85.165756
Train Epoch: 155 [3328/60000 (6%)]	Loss: 86.520477
Train Epoch: 155 [3584/60000 (6%)]	Loss: 183.425659
Train Epoch: 155 [3840/60000 (6%)]	Loss: 86.553078
Train Epoch: 155 [4096/60000 (7%)]	Loss: 88.866219
Train Epoch: 155 [4352/60000 (7%)]	Loss: 99.568756
Train Epoch: 155 [4608/60000 (8%)]	Loss: 118.749146
Train Epoch: 155 [4864/60000 (8%)]	Loss: 82.694206
Train Epoch: 155 [5120/60000 (9%)]	Loss: 85.651772
Train Epoch: 155 [5376/60000 (9%)]	Loss: 83.573853
Train Epoch: 155 [5632/60000 (9%)]	Loss: 170.618576
Train Epoch: 155 [5888/60000 (10%)]	Loss: 104.852219
Train Epoch: 155 [6144/60000 (10%)]	Loss: 84.759094
Train Epoch: 155 [6400/60000 (11%)]	Loss: 83.788292
Train Epoch: 155 [6656

Train Epoch: 155 [41472/60000 (69%)]	Loss: 80.816284
Train Epoch: 155 [41728/60000 (69%)]	Loss: 114.866264
Train Epoch: 155 [41984/60000 (70%)]	Loss: 90.338074
Train Epoch: 155 [42240/60000 (70%)]	Loss: 78.806328
Train Epoch: 155 [42496/60000 (71%)]	Loss: 83.384499
Train Epoch: 155 [42752/60000 (71%)]	Loss: 81.400764
Train Epoch: 155 [43008/60000 (71%)]	Loss: 82.721474
Train Epoch: 155 [43264/60000 (72%)]	Loss: 83.901810
Train Epoch: 155 [43520/60000 (72%)]	Loss: 89.511581
Train Epoch: 155 [43776/60000 (73%)]	Loss: 101.639626
Train Epoch: 155 [44032/60000 (73%)]	Loss: 78.161789
Train Epoch: 155 [44288/60000 (74%)]	Loss: 101.545090
Train Epoch: 155 [44544/60000 (74%)]	Loss: 82.056023
Train Epoch: 155 [44800/60000 (74%)]	Loss: 115.822533
Train Epoch: 155 [45056/60000 (75%)]	Loss: 85.609932
Train Epoch: 155 [45312/60000 (75%)]	Loss: 81.471115
Train Epoch: 155 [45568/60000 (76%)]	Loss: 80.045822
Train Epoch: 155 [45824/60000 (76%)]	Loss: 80.896049
Train Epoch: 155 [46080/60000 (77%)]	Loss:

Train Epoch: 156 [20992/60000 (35%)]	Loss: 82.102654
Train Epoch: 156 [21248/60000 (35%)]	Loss: 94.737892
Train Epoch: 156 [21504/60000 (36%)]	Loss: 84.607185
Train Epoch: 156 [21760/60000 (36%)]	Loss: 91.502884
Train Epoch: 156 [22016/60000 (37%)]	Loss: 82.654388
Train Epoch: 156 [22272/60000 (37%)]	Loss: 85.916245
Train Epoch: 156 [22528/60000 (37%)]	Loss: 81.494125
Train Epoch: 156 [22784/60000 (38%)]	Loss: 83.648926
Train Epoch: 156 [23040/60000 (38%)]	Loss: 133.239273
Train Epoch: 156 [23296/60000 (39%)]	Loss: 80.925819
Train Epoch: 156 [23552/60000 (39%)]	Loss: 100.444130
Train Epoch: 156 [23808/60000 (40%)]	Loss: 84.124298
Train Epoch: 156 [24064/60000 (40%)]	Loss: 115.682083
Train Epoch: 156 [24320/60000 (40%)]	Loss: 99.546974
Train Epoch: 156 [24576/60000 (41%)]	Loss: 83.867798
Train Epoch: 156 [24832/60000 (41%)]	Loss: 81.923401
Train Epoch: 156 [25088/60000 (42%)]	Loss: 86.842911
Train Epoch: 156 [25344/60000 (42%)]	Loss: 86.149307
Train Epoch: 156 [25600/60000 (43%)]	Loss: 

Train Epoch: 157 [0/60000 (0%)]	Loss: 95.840103
Train Epoch: 157 [256/60000 (0%)]	Loss: 131.487976
Train Epoch: 157 [512/60000 (1%)]	Loss: 82.057777
Train Epoch: 157 [768/60000 (1%)]	Loss: 88.351196
Train Epoch: 157 [1024/60000 (2%)]	Loss: 87.164658
Train Epoch: 157 [1280/60000 (2%)]	Loss: 81.305458
Train Epoch: 157 [1536/60000 (3%)]	Loss: 83.190155
Train Epoch: 157 [1792/60000 (3%)]	Loss: 83.687271
Train Epoch: 157 [2048/60000 (3%)]	Loss: 82.626343
Train Epoch: 157 [2304/60000 (4%)]	Loss: 136.004517
Train Epoch: 157 [2560/60000 (4%)]	Loss: 96.372055
Train Epoch: 157 [2816/60000 (5%)]	Loss: 79.963249
Train Epoch: 157 [3072/60000 (5%)]	Loss: 109.567329
Train Epoch: 157 [3328/60000 (6%)]	Loss: 87.882629
Train Epoch: 157 [3584/60000 (6%)]	Loss: 142.507584
Train Epoch: 157 [3840/60000 (6%)]	Loss: 87.586563
Train Epoch: 157 [4096/60000 (7%)]	Loss: 83.576698
Train Epoch: 157 [4352/60000 (7%)]	Loss: 126.543098
Train Epoch: 157 [4608/60000 (8%)]	Loss: 82.936920
Train Epoch: 157 [4864/60000 (8%

KeyboardInterrupt: 

In [17]:
torch.save(model.state_dict(), result_path+"model.pt")

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=5e-4)
epochs=200

In [10]:

model.load_state_dict(torch.load(result_path+"model.pt"))
model.eval()

NP(
  (h_1): Linear(in_features=3, out_features=400, bias=True)
  (h_2): Linear(in_features=400, out_features=400, bias=True)
  (h_3): Linear(in_features=400, out_features=128, bias=True)
  (latent_layer_1): Linear(in_features=3, out_features=400, bias=True)
  (latent_layer_2): Linear(in_features=400, out_features=400, bias=True)
  (latent_layer_3): Linear(in_features=400, out_features=128, bias=True)
  (s_to_z_mean): Linear(in_features=128, out_features=128, bias=True)
  (s_to_z_logvar): Linear(in_features=128, out_features=128, bias=True)
  (g_1): Linear(in_features=258, out_features=400, bias=True)
  (g_2): Linear(in_features=400, out_features=400, bias=True)
  (g_3): Linear(in_features=400, out_features=400, bias=True)
  (g_y): Linear(in_features=400, out_features=1, bias=True)
)

In [12]:
for epoch in range(157, epochs + 157):
    
    test(epoch)
    train(epoch)

/home/h86479/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


KeyboardInterrupt: 